In [ ]:




import pandas as pd
import numpy as np
from pathlib import Path


#BASE_DIR = Path(r"/path/to/your/cchain_folder")  # <<< CHANGE THIS

LOCATION_FILE    = "location.csv"              # the file you uploaded
WORLDPOP_FILE    = "worldpop_population.csv"
PSA_FILE         = "disease_psa_totals.csv"
NIGHTLIGHTS_FILE = "nighttime_lights.csv"
HEALTH_FILE      = "osm_poi_health.csv"     # file with hospital_nearest
RWI_FILE         = "tm_relative_wealth_index.csv"
CLIMATE_FILE     = "climate_atmosphere.csv"
AIRQ_FILE        = "climate_air_quality.csv"

YEAR_MIN = 2018
YEAR_MAX = 2021

# ============================================================
# COLUMN NAMES (BASED ON YOUR DESCRIPTION)
# ============================================================

COL_CITY_ID   = "adm3_pcode"
COL_ADM4      = "adm4_pcode"
COL_DATE      = "date"
COL_YEAR      = "year"

# WorldPop
COL_POP_TOTAL = "pop_count_total"

# PSA
COL_DISEASE_COMMON = "disease_common_name"
COL_DEATH_TOTAL    = "death_total"
DIABETES_KEYWORD   = "diabet"   # will match "DIABETES", "Diabetes Mellitus", etc. (case-insensitive)

# Nightlights
COL_NTL_MEAN = "avg_rad_mean"

# Health facilities
COL_HOSP_NEAREST = "hospital_nearest"

# Wealth index
COL_RWI_MEAN = "rwi_mean"

# Climate
COL_TAVE = "tave"   # daily average temp

# Air quality
COL_PM25 = "pm25"

# ============================================================
# LOCATION MAPPING (adm4 → adm3 + city_name)
# ============================================================

_location_map = None

def load_location():
    """
    Load location.csv and return mapping:
    adm4_pcode → adm3_pcode + city_name
    """
    global _location_map
    if _location_map is None:
        loc = pd.read_csv(LOCATION_FILE)
        # adjust these three column names if needed
        loc = loc[[COL_ADM4, COL_CITY_ID, "adm3_en"]].drop_duplicates()
        loc = loc.rename(columns={"adm3_en": "city_name"})
        _location_map = loc
    return _location_map


def ensure_city_id(df, df_name="df"):
    """
    Ensure df has COL_CITY_ID. If not, but has adm4_pcode, merge from location.csv.
    """
    if COL_CITY_ID in df.columns:
        return df
    if COL_ADM4 in df.columns:
        loc = load_location()
        df = df.merge(loc[[COL_ADM4, COL_CITY_ID]], on=COL_ADM4, how="left")
        return df
    raise KeyError(f"{df_name} has neither '{COL_CITY_ID}' nor '{COL_ADM4}'.")


def restrict_years(df, year_col=COL_YEAR, year_min=YEAR_MIN, year_max=YEAR_MAX):
    return df[(df[year_col] >= year_min) & (df[year_col] <= year_max)].copy()

# ============================================================
# 1. Population: city-level pop_mean from WorldPop (2018–2020)
# ============================================================

def build_city_pop_mean():
    """
    Compute city-level mean population (pop_mean) using WorldPop.
    WorldPop has adm4 + date; we:
    - map adm4 → adm3
    - extract year from date
    - keep 2018–2020
    - sum pop_count_total per city-year
    - average across years → pop_mean per city
    """
    pop = pd.read_csv(WORLDPOP_FILE)
    pop = ensure_city_id(pop, df_name="WorldPop")

    pop[COL_DATE] = pd.to_datetime(pop[COL_DATE], errors="coerce", dayfirst=True)
    pop[COL_YEAR] = pop[COL_DATE].dt.year

    pop = pop[(pop[COL_YEAR] >= 2018) & (pop[COL_YEAR] <= 2020)].copy()

    pop_city_year = (
        pop.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_POP_TOTAL]
           .sum()
           .rename(columns={COL_POP_TOTAL: "pop_total"})
    )

    pop_city_mean = (
        pop_city_year.groupby(COL_CITY_ID, as_index=False)["pop_total"]
                     .mean()
                     .rename(columns={"pop_total": "pop_mean"})
    )

    return pop_city_mean

# ============================================================
# 2. PSA diabetes deaths → diab_death_rate
# ============================================================

def build_psa_deaths(pop_city):
    """
    Build city-year diabetes deaths and death rate per 100k using PSA.
    """
    psa = pd.read_csv(PSA_FILE)
    psa = ensure_city_id(psa, df_name="PSA")

    psa[COL_DATE] = pd.to_datetime(psa[COL_DATE], errors="coerce", dayfirst=True)
    psa[COL_YEAR] = psa[COL_DATE].dt.year

    psa = restrict_years(psa, COL_YEAR)

    # filter disease_common_name containing "diabet"
    psa = psa[psa[COL_DISEASE_COMMON].str.contains(DIABETES_KEYWORD,
                                                   case=False,
                                                   na=False)].copy()

    deaths_city_year = (
        psa.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_DEATH_TOTAL]
           .sum()
           .rename(columns={COL_DEATH_TOTAL: "diab_deaths"})
    )

    deaths_city_year = deaths_city_year.merge(pop_city, on=COL_CITY_ID, how="left")

    deaths_city_year["diab_death_rate"] = (
        deaths_city_year["diab_deaths"] / deaths_city_year["pop_mean"] * 100_000
    )

    return deaths_city_year

# ============================================================
# 3. Nightlights → nightlights_log
# ============================================================

def build_nightlights():
    """
    Build city-year nightlights_log from nighttime_lights (adm4-level yearly).
    """
    nl = pd.read_csv(NIGHTLIGHTS_FILE)
    nl = ensure_city_id(nl, df_name="Nightlights")

    # if year isn't present but date is, derive it
    if COL_YEAR not in nl.columns and COL_DATE in nl.columns:
        nl[COL_DATE] = pd.to_datetime(nl[COL_DATE], errors="coerce", dayfirst=True)
        nl[COL_YEAR] = nl[COL_DATE].dt.year

    nl = restrict_years(nl, COL_YEAR)

    nl_city_year = (
        nl.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_NTL_MEAN]
          .mean()
          .rename(columns={COL_NTL_MEAN: "nightlights_mean"})
    )

    eps = 1e-6
    nl_city_year["nightlights_log"] = np.log(
        nl_city_year["nightlights_mean"].clip(lower=0) + eps
    )

    return nl_city_year[[COL_CITY_ID, COL_YEAR, "nightlights_log"]]

# ============================================================
# 4. Health access → hospital_nearest_mean
# ============================================================

def build_health_access():
    """
    Build city-year mean distance/time to nearest hospital (hospital_nearest_mean).
    """
    hf = pd.read_csv(HEALTH_FILE)
    hf = ensure_city_id(hf, df_name="Health facilities")

    if COL_YEAR not in hf.columns and COL_DATE in hf.columns:
        hf[COL_DATE] = pd.to_datetime(hf[COL_DATE], errors="coerce", dayfirst=True)
        hf[COL_YEAR] = hf[COL_DATE].dt.year

    hf = restrict_years(hf, COL_YEAR)

    hf_city_year = (
        hf.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_HOSP_NEAREST]
          .mean()
          .rename(columns={COL_HOSP_NEAREST: "hospital_nearest_mean"})
    )

    return hf_city_year[[COL_CITY_ID, COL_YEAR, "hospital_nearest_mean"]]

# ============================================================
# 5. Wealth index → wealth_index_city + high_poverty
# ============================================================

def build_wealth_and_dummy():
    """
    Build city-level wealth_index_city (mean rwi_mean) and high_poverty dummy.
    """
    rwi = pd.read_csv(RWI_FILE)
    rwi = ensure_city_id(rwi, df_name="RWI")

    city_wealth = (
        rwi.groupby(COL_CITY_ID, as_index=False)[COL_RWI_MEAN]
           .mean()
           .rename(columns={COL_RWI_MEAN: "wealth_index_city"})
    )

    median_val = city_wealth["wealth_index_city"].median()
    city_wealth["high_poverty"] = (
        city_wealth["wealth_index_city"] < median_val
    ).astype(int)

    return city_wealth[[COL_CITY_ID, "wealth_index_city", "high_poverty"]]

# ============================================================
# 6. Climate → mean_temp_c (tave)
# ============================================================

def build_mean_temp():
    """
    Build city-year mean_temp_c from climate_atmosphere (daily tave, adm4).
    """
    clim = pd.read_csv(CLIMATE_FILE)
    clim = ensure_city_id(clim, df_name="Climate")

    clim[COL_DATE] = pd.to_datetime(clim[COL_DATE], errors="coerce", dayfirst=True)
    clim[COL_YEAR] = clim[COL_DATE].dt.year
    clim = restrict_years(clim, COL_YEAR)

    temp_city_year = (
        clim.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_TAVE]
             .mean()
             .rename(columns={COL_TAVE: "mean_temp_c"})
    )

    return temp_city_year[[COL_CITY_ID, COL_YEAR, "mean_temp_c"]]

# ============================================================
# 7. Air quality → pm25_mean
# ============================================================

def build_pm25():
    """
    Build city-year pm25_mean from climate_air_quality (daily pm25, adm4).
    """
    air = pd.read_csv(AIRQ_FILE)
    air = ensure_city_id(air, df_name="Air quality")

    air[COL_DATE] = pd.to_datetime(air[COL_DATE], errors="coerce", dayfirst=True)
    air[COL_YEAR] = air[COL_DATE].dt.year
    air = restrict_years(air, COL_YEAR)

    pm_city_year = (
        air.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_PM25]
           .mean()
           .rename(columns={COL_PM25: "pm25_mean"})
    )

    return pm_city_year[[COL_CITY_ID, COL_YEAR, "pm25_mean"]]

# ============================================================
# 8. Build final dataset (city-year 2018–2021)
# ============================================================

def build_final_dataset():
    # City-level pop_mean
    pop_city = build_city_pop_mean()

    # Core pieces
    deaths = build_psa_deaths(pop_city)   # diab_deaths, diab_death_rate, pop_mean
    night  = build_nightlights()          # nightlights_log
    temp   = build_mean_temp()            # mean_temp_c
    pm25   = build_pm25()                 # pm25_mean
    wealth = build_wealth_and_dummy()     # wealth_index_city, high_poverty
    hosp   = build_health_access()        # hospital_nearest_mean

    # Start from deaths
    df = deaths.copy()

    # Merge city-year predictors
    for other in [night, temp, pm25, hosp]:
        df = df.merge(other, on=[COL_CITY_ID, COL_YEAR], how="left")

    # Merge city-level predictors
    df = df.merge(wealth, on=COL_CITY_ID, how="left")

    # Add city_name from location
    loc = load_location()[[COL_CITY_ID, "city_name"]].drop_duplicates()
    df = df.merge(loc, on=COL_CITY_ID, how="left")

    df = restrict_years(df, COL_YEAR)
    df = df[~df["diab_death_rate"].isna()].copy()

    final_cols = [
        COL_CITY_ID,
        "city_name",
        COL_YEAR,
        "diab_deaths",
        "diab_death_rate",
        "pop_mean",
        "nightlights_log",
        "mean_temp_c",
        "pm25_mean",
        "hospital_nearest_mean",
        "wealth_index_city",
        "high_poverty",
    ]

    missing = [c for c in final_cols if c not in df.columns]
    if missing:
        print("Warning: these expected columns are missing:", missing)
        print("Available columns:", df.columns.tolist())

    df_final = df[[c for c in final_cols if c in df.columns]]

    return df_final

# ============================================================
# MAIN
# ============================================================








if __name__ == "__main__":
    print("Final dataset:")
    df_final = build_final_dataset()
    print(df_final.head())
    print("Shape:", df_final.shape)

    out_path = "diabetes_regression_city_year_2018_2021.csv"
    df_final.to_csv(out_path, index=False)
    print("Saved dataset to:", out_path)


Building final diabetes regression dataset (city-year 2018–2021)...
    adm3_pcode     city_name    year  diab_deaths  diab_death_rate  \
0  PH015518000  Dagupan City  2018.0           95        46.903383   
1  PH015518000  Dagupan City  2019.0           87        42.953625   
2  PH015518000  Dagupan City  2020.0          100        49.371982   
3  PH015518000  Dagupan City  2021.0          101        49.865702   
4  PH034919000  Palayan City  2018.0            3         5.618694   

        pop_mean  nightlights_log  mean_temp_c  pm25_mean  \
0  202544.024373         1.902008    27.692509  18.295625   
1  202544.024373         1.920269    27.569729  19.399861   
2  202544.024373         1.909537    28.005887  18.576250   
3  202544.024373         1.849594    27.952760  19.580903   
4   53393.188070        -0.166533    26.663969  28.506042   

   hospital_nearest_mean  wealth_index_city  high_poverty  
0             458.801367           0.636998             0  
1             458.801367

In [6]:
'''import pandas as pd
import numpy as np
from pathlib import Path

# ============================================================
# CONFIG – EDIT THESE TO MATCH YOUR FOLDER / FILENAMES
# ============================================================


LOCATION_FILE    = "location.csv"              # the file you uploaded
WORLDPOP_FILE    = "worldpop_population.csv"
PSA_FILE         = "disease_psa_totals.csv"
NIGHTLIGHTS_FILE = "nighttime_lights.csv"
HEALTH_FILE      = "health_facilities.csv"     # file with hospital_nearest
RWI_FILE         = "tm_relative_wealth_index.csv"
CLIMATE_FILE     = "climate_atmosphere.csv"
AIRQ_FILE        = "climate_air_quality.csv"

YEAR_MIN = 2018
YEAR_MAX = 2021

# ============================================================
# COLUMN NAMES (MATCHING YOUR DESCRIPTION)
# ============================================================

COL_CITY_ID   = "adm3_pcode"
COL_ADM4      = "adm4_pcode"
COL_DATE      = "date"
COL_YEAR      = "year"

# WorldPop
COL_POP_TOTAL = "pop_count_total"

# PSA
COL_DISEASE_COMMON = "disease_common_name"
COL_DEATH_TOTAL    = "death_total"
DIABETES_KEYWORD   = "diabet"   # match "diabet" in disease_common_name (case-insensitive)

# Nightlights
COL_NTL_MEAN = "avg_rad_mean"

# Health facilities
COL_HOSP_NEAREST = "hospital_nearest"

# Wealth index
COL_RWI_MEAN = "rwi_mean"

# Climate
COL_TAVE = "tave"   # daily average temp

# Air quality
COL_PM25 = "pm25"


# ============================================================
# LOCATION MAPPING (adm4 → adm3 + city_name)
# ============================================================

_location_map = None

def load_location():
    """
    Load location.csv and return a mapping:
    adm4_pcode → adm3_pcode + city_name
    """
    global _location_map
    if _location_map is None:
        loc = pd.read_csv(LOCATION_FILE)
        # Expect columns like: adm3_pcode, adm3_en, adm4_pcode, ...
        loc = loc[[COL_ADM4, COL_CITY_ID, "adm3_en"]].drop_duplicates()
        loc = loc.rename(columns={"adm3_en": "city_name"})
        _location_map = loc
    return _location_map


def ensure_city_id(df, df_name="df"):
    """
    Make sure df has COL_CITY_ID. If not, but it has adm4_pcode,
    merge from location.csv.
    """
    if COL_CITY_ID in df.columns:
        return df
    if COL_ADM4 in df.columns:
        loc = load_location()
        df = df.merge(loc[[COL_ADM4, COL_CITY_ID]], on=COL_ADM4, how="left")
        return df
    raise KeyError(f"{df_name} has neither '{COL_CITY_ID}' nor '{COL_ADM4}'.")


def restrict_years(df, year_col=COL_YEAR, year_min=YEAR_MIN, year_max=YEAR_MAX):
    return df[(df[year_col] >= year_min) & (df[year_col] <= year_max)].copy()


# ============================================================
# 1. Population: city-level pop_mean from WorldPop (2018–2020)
# ============================================================

def build_city_pop_mean():
    """
    Use WorldPop (adm4, daily) to compute a city-level mean population (pop_mean)
    over 2018–2020. We treat this as constant population for 2018–2021.
    """
    pop = pd.read_csv(WORLDPOP_FILE, parse_dates=[COL_DATE])
    # Ensure we have city id
    pop = ensure_city_id(pop, df_name="WorldPop")

    # Create 'year' from 'date'
    pop[COL_YEAR] = pop[COL_DATE].dt.year

    # Keep only 2018–2020
    pop = pop[(pop[COL_YEAR] >= 2018) & (pop[COL_YEAR] <= 2020)].copy()

    # Sum barangay (adm4) populations to city-year totals
    pop_city_year = (
        pop.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_POP_TOTAL]
           .sum()
           .rename(columns={COL_POP_TOTAL: "pop_total"})
    )

    # Average over years per city → pop_mean
    pop_city_mean = (
        pop_city_year.groupby(COL_CITY_ID, as_index=False)["pop_total"]
                     .mean()
                     .rename(columns={"pop_total": "pop_mean"})
    )

    return pop_city_mean


# ============================================================
# 2. PSA diabetes deaths → diab_death_rate
# ============================================================

def build_psa_deaths(pop_city):
    """
    Build city-year diabetes deaths and death rate per 100k using PSA.
    """
    psa = pd.read_csv(PSA_FILE, parse_dates=[COL_DATE], dayfirst=True)
    # Expect: adm3_pcode, date, disease_common_name, death_total, ...

    psa = ensure_city_id(psa, df_name="PSA")
    psa[COL_YEAR] = psa[COL_DATE].dt.year
    psa = restrict_years(psa, COL_YEAR)

    # Filter rows where disease_common_name contains "diabet"
    psa = psa[psa[COL_DISEASE_COMMON].str.contains(DIABETES_KEYWORD,
                                                   case=False,
                                                   na=False)].copy()

    # Sum weekly deaths into city-year totals
    deaths_city_year = (
        psa.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_DEATH_TOTAL]
           .sum()
           .rename(columns={COL_DEATH_TOTAL: "diab_deaths"})
    )

    # Merge pop_mean per city
    deaths_city_year = deaths_city_year.merge(pop_city, on=COL_CITY_ID, how="left")

    # Compute death rate per 100,000
    deaths_city_year["diab_death_rate"] = (
        deaths_city_year["diab_deaths"] / deaths_city_year["pop_mean"] * 100_000
    )

    return deaths_city_year


# ============================================================
# 3. Nightlights → nightlights_log
# ============================================================

def build_nightlights():
    """
    Build city-year nightlights_log from nighttime_lights (adm4, yearly).
    """
    nl = pd.read_csv(NIGHTLIGHTS_FILE)
    # Expect: adm4_pcode, year, avg_rad_mean, ...

    nl = ensure_city_id(nl, df_name="Nightlights")
    if COL_YEAR not in nl.columns and COL_DATE in nl.columns:
        nl[COL_DATE] = pd.to_datetime(nl[COL_DATE])
        nl[COL_YEAR] = nl[COL_DATE].dt.year

    nl = restrict_years(nl, COL_YEAR)

    nl_city_year = (
        nl.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_NTL_MEAN]
          .mean()
          .rename(columns={COL_NTL_MEAN: "nightlights_mean"})
    )

    eps = 1e-6
    nl_city_year["nightlights_log"] = np.log(
        nl_city_year["nightlights_mean"].clip(lower=0) + eps
    )

    return nl_city_year[[COL_CITY_ID, COL_YEAR, "nightlights_log"]]


# ============================================================
# 4. Health access → hospital_nearest_mean
# ============================================================

def build_health_access():
    """
    Build city-year mean distance/time to nearest hospital (hospital_nearest_mean).
    """
    hf = pd.read_csv(HEALTH_FILE)
    # Expect: adm4_pcode, year, hospital_nearest, ...

    hf = ensure_city_id(hf, df_name="Health facilities")

    if COL_YEAR not in hf.columns and COL_DATE in hf.columns:
        hf[COL_DATE] = pd.to_datetime(hf[COL_DATE])
        hf[COL_YEAR] = hf[COL_DATE].dt.year

    hf = restrict_years(hf, COL_YEAR)

    hf_city_year = (
        hf.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_HOSP_NEAREST]
          .mean()
          .rename(columns={COL_HOSP_NEAREST: "hospital_nearest_mean"})
    )

    return hf_city_year[[COL_CITY_ID, COL_YEAR, "hospital_nearest_mean"]]


# ============================================================
# 5. Wealth index → wealth_index_city + high_poverty
# ============================================================

def build_wealth_and_dummy():
    """
    Build city-level wealth_index_city (mean rwi_mean) and high_poverty dummy.
    """
    rwi = pd.read_csv(RWI_FILE)
    # Expect: adm4_pcode, rwi_mean, ...

    rwi = ensure_city_id(rwi, df_name="RWI")

    city_wealth = (
        rwi.groupby(COL_CITY_ID, as_index=False)[COL_RWI_MEAN]
           .mean()
           .rename(columns={COL_RWI_MEAN: "wealth_index_city"})
    )

    median_val = city_wealth["wealth_index_city"].median()
    city_wealth["high_poverty"] = (
        city_wealth["wealth_index_city"] < median_val
    ).astype(int)

    return city_wealth[[COL_CITY_ID, "wealth_index_city", "high_poverty"]]


# ============================================================
# 6. Climate → mean_temp_c (tave)
# ============================================================

def build_mean_temp():
    """
    Build city-year mean_temp_c from climate_atmosphere (daily tave, adm4).
    """
    clim = pd.read_csv(CLIMATE_FILE, parse_dates=[COL_DATE], dayfirst=True)
    # Expect: adm4_pcode, date, tave, ...

    clim = ensure_city_id(clim, df_name="Climate")
    clim[COL_YEAR] = clim[COL_DATE].dt.year
    clim = restrict_years(clim, COL_YEAR)

    temp_city_year = (
        clim.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_TAVE]
             .mean()
             .rename(columns={COL_TAVE: "mean_temp_c"})
    )

    return temp_city_year[[COL_CITY_ID, COL_YEAR, "mean_temp_c"]]


# ============================================================
# 7. Air quality → pm25_mean
# ============================================================

def build_pm25():
    """
    Build city-year pm25_mean from climate_air_quality (daily pm25, adm4).
    """
    air = pd.read_csv(AIRQ_FILE, parse_dates=[COL_DATE], dayfirst=True)
    # Expect: adm4_pcode, date, pm25, ...

    air = ensure_city_id(air, df_name="Air quality")
    air[COL_YEAR] = air[COL_DATE].dt.year
    air = restrict_years(air, COL_YEAR)

    pm_city_year = (
        air.groupby([COL_CITY_ID, COL_YEAR], as_index=False)[COL_PM25]
           .mean()
           .rename(columns={COL_PM25: "pm25_mean"})
    )

    return pm_city_year[[COL_CITY_ID, COL_YEAR, "pm25_mean"]]


# ============================================================
# 8. Build final dataset (city-year 2018–2021)
# ============================================================

def build_final_dataset():
    # City-level pop_mean
    pop_city = build_city_pop_mean()

    # Core pieces
    deaths = build_psa_deaths(pop_city)   # diab_deaths, diab_death_rate, pop_mean
    night  = build_nightlights()          # nightlights_log
    temp   = build_mean_temp()            # mean_temp_c
    pm25   = build_pm25()                 # pm25_mean
    wealth = build_wealth_and_dummy()     # wealth_index_city, high_poverty
    hosp   = build_health_access()        # hospital_nearest_mean

    # Start from deaths
    df = deaths.copy()

    # Merge city-year predictors
    for other in [night, temp, pm25, hosp]:
        df = df.merge(other, on=[COL_CITY_ID, COL_YEAR], how="left")

    # Merge city-level predictors
    df = df.merge(wealth, on=COL_CITY_ID, how="left")

    # Add city_name from location
    loc = load_location()[[COL_CITY_ID, "city_name"]].drop_duplicates()
    df = df.merge(loc, on=COL_CITY_ID, how="left")

    # Keep only rows with non-missing death rate
    df = restrict_years(df, COL_YEAR)
    df = df[~df["diab_death_rate"].isna()].copy()

    # Final columns
    final_cols = [
        COL_CITY_ID,
        "city_name",
        COL_YEAR,
        "diab_deaths",
        "diab_death_rate",
        "pop_mean",
        "nightlights_log",
        "mean_temp_c",
        "pm25_mean",
        "hospital_nearest_mean",
        "wealth_index_city",
        "high_poverty",
    ]

    missing = [c for c in final_cols if c not in df.columns]
    if missing:
        print("Warning: these expected columns are missing:", missing)
        print("Available columns:", df.columns.tolist())

    df_final = df[[c for c in final_cols if c in df.columns]]

    return df_final


# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":
    print("Building final diabetes regression dataset (city-year 2018–2021)...")
    df_final = build_final_dataset()
    print(df_final.head())
    print("Shape:", df_final.shape)

    out_path = BASE_DIR / "diabetes_regression_city_year_2018_2021.csv"
    df_final.to_csv(out_path, index=False)
    print("Saved dataset to:", out_path)
'''

'import pandas as pd\nimport numpy as np\nfrom pathlib import Path\n\n# ============================================================\n# CONFIG – EDIT THESE TO MATCH YOUR FOLDER / FILENAMES\n# ============================================================\n\n\nLOCATION_FILE    = "location.csv"              # the file you uploaded\nWORLDPOP_FILE    = "worldpop_population.csv"\nPSA_FILE         = "disease_psa_totals.csv"\nNIGHTLIGHTS_FILE = "nighttime_lights.csv"\nHEALTH_FILE      = "health_facilities.csv"     # file with hospital_nearest\nRWI_FILE         = "tm_relative_wealth_index.csv"\nCLIMATE_FILE     = "climate_atmosphere.csv"\nAIRQ_FILE        = "climate_air_quality.csv"\n\nYEAR_MIN = 2018\nYEAR_MAX = 2021\n\n# ============================================================\n# COLUMN NAMES (MATCHING YOUR DESCRIPTION)\n# ============================================================\n\nCOL_CITY_ID   = "adm3_pcode"\nCOL_ADM4      = "adm4_pcode"\nCOL_DATE      = "date"\nCOL_YEAR      = "year